In [9]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='roberta-large')

result = unmasker(["Water is <mask>", "Air is <mask>"])

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-large and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
result[0][0]['sequence'].replace('<s>', '').replace('</s>', '')

'Water is life'

In [16]:
import pandas as pd
from transformers import pipeline
unmasker = pipeline('fill-mask', model='roberta-large')

df_m = pd.read_json('/nas/home/yixiaoli/prompt_commonsense/Com2Sense/datasets/com2sense/m/dev_m.json')
df_or = pd.read_json('/nas/home/yixiaoli/prompt_commonsense/Com2Sense/datasets/com2sense/or/dev_or.json')

df_pattern_1 = pd.DataFrame(columns = df_m.columns)
df_pattern_2 = pd.DataFrame(columns = df_m.columns)


for (index, row_m), (_, row_or) in zip(df_m.iterrows(), df_or.iterrows()):
    print(index)

    # extract the keywords from Modified Dataset 
    # and original sentence from Original_removed Dataset
    keywords_1 = row_m['sent_1'].split('.')[0].split(', ')     ## ['two people', 'a group']
    keywords_2 = row_m['sent_2'].split('.')[0].split(', ')
    sentence_1 = row_or['sent_1']
    sentence_2 = row_or['sent_2']
    
    # initial the supporting sentence pattern
    pattern_1_1 = f"{keywords_1[0]} is <mask> than {keywords_1[1]}."
    pattern_1_2 = f"{keywords_2[0]} is <mask> than {keywords_2[1]}."
    pattern_2_1 = f"{keywords_1[0]} has more <mask> than {keywords_1[1]}."
    pattern_2_2 = f"{keywords_2[0]} has more <mask> than {keywords_2[1]}."
    
    # predict the mask
    model_input = [pattern_1_1, pattern_1_2, pattern_2_1, pattern_2_2]
    result = unmasker(model_input)
    
    # extract the full supporting sentences
    pattern_1_1 = result[0][0]['sequence'].replace('<s>', '').replace('</s>', '')
    pattern_1_2 = result[1][0]['sequence'].replace('<s>', '').replace('</s>', '')
    pattern_2_1 = result[2][0]['sequence'].replace('<s>', '').replace('</s>', '')
    pattern_2_2 = result[3][0]['sequence'].replace('<s>', '').replace('</s>', '')
    
    # add the supporting sentence to original sentence
    
    ## pattern 1
    row_or['sent_1'] = f"{pattern_1_1} {sentence_1}"
    row_or['sent_2'] = f"{pattern_1_2} {sentence_2}"
    df_pattern_1 = df_pattern_1.append(row_or)
    
    ## pattern 2
    row_or['sent_1'] = f"{pattern_2_1} {sentence_1}"
    row_or['sent_2'] = f"{pattern_2_2} {sentence_2}"
    df_pattern_2 = df_pattern_2.append(row_or)
    
    
df_pattern_1.to_json('/nas/home/yixiaoli/prompt_commonsense/com_play/datasets/com2sense/dev_s1.json', orient='records')
df_pattern_2.to_json('/nas/home/yixiaoli/prompt_commonsense/com_play/datasets/com2sense/dev_s2.json', orient='records')



Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-large and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [15]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

sentence = 'summarize: compared with short glasses, tall glasses '

input_ids = tokenizer.encode(sentence, return_tensors='pt')
greedy_output = model.generate(input_ids, num_beams=7, no_repeat_ngram_size=2, min_length=16, max_length=24)
print("Output:\n" + 100 * '-')

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
compared with short glasses, tall glasses have a longer lifespan.


In [42]:
import pandas as pd
from transformers import pipeline
unmasker = pipeline('fill-mask', model='roberta-large')

df_m = pd.read_json('/nas/home/yixiaoli/prompt_commonsense/Com2Sense/datasets/com2sense/m/test_m.json')
df_or = pd.read_json('/nas/home/yixiaoli/prompt_commonsense/Com2Sense/datasets/com2sense/or/test_or.json')

df_m = df_m[df_m['scenario']=='comparison']
df_or = df_or[df_or['scenario']=='comparison']

df_p = pd.DataFrame(columns = df_m.columns)


tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained("t5-large")

for (index, row_m), (_, row_or) in zip(df_m.iterrows(), df_or.iterrows()):
    print(index)

    # extract the keywords from Modified Dataset 
    # and original sentence from Original_removed Dataset
    keywords_1 = row_m['sent_1'].split('.')[0].split(', ')     ## ['two people', 'a group']
    keywords_2 = row_m['sent_2'].split('.')[0].split(', ')
    print(keywords_1)
    sentence_1 = row_or['sent_1']
    sentence_2 = row_or['sent_2']
    
    # initial the supporting sentence pattern
    prompt_1_1 = f"summarize: Compared with {keywords_1[0]}, {keywords_1[1]}"
    prompt_1_2 = f"summarize: Compared with {keywords_1[1]}, {keywords_1[0]}"
    prompt_2_1 = f"summarize: Compared with {keywords_2[0]}, {keywords_2[1]}"
    prompt_2_2 = f"summarize: Compared with {keywords_2[1]}, {keywords_2[0]}" 
    
    # generate the sentence 
    input_1_1 = tokenizer.encode(prompt_1_1, return_tensors='pt')
    input_1_2 = tokenizer.encode(prompt_1_2, return_tensors='pt')
    input_2_1 = tokenizer.encode(prompt_2_1, return_tensors='pt')
    input_2_2 = tokenizer.encode(prompt_2_2, return_tensors='pt')
    
    output_1_1 = model.generate(input_1_1, num_beams=8, no_repeat_ngram_size=2, min_length=24, max_length=36)
    output_1_2 = model.generate(input_1_2, num_beams=8, no_repeat_ngram_size=2, min_length=24, max_length=36)
    output_2_1 = model.generate(input_2_1, num_beams=8, no_repeat_ngram_size=2, min_length=24, max_length=36)
    output_2_2 = model.generate(input_2_2, num_beams=8, no_repeat_ngram_size=2, min_length=24, max_length=36)
    
    # get the full supporting sentences
    prompt_1_1 = tokenizer.decode(output_1_1[0], skip_special_tokens=True)
    prompt_1_2 = tokenizer.decode(output_1_2[0], skip_special_tokens=True)
    prompt_2_1 = tokenizer.decode(output_2_1[0], skip_special_tokens=True)
    prompt_2_2 = tokenizer.decode(output_2_2[0], skip_special_tokens=True)
    
    prompt_1 = prompt_1_1 + ' Meanwhile, ' + prompt_1_2
    prompt_2 = prompt_2_1 + ' Meanwhile, ' + prompt_2_2
    print(prompt_1)
    
    # add the supporting sentence to original sentence
    row_or['sent_1'] = f"{prompt_1} {sentence_1}"
    row_or['sent_2'] = f"{prompt_2} {sentence_2}"
    df_p = df_p.append(row_or)
    
df_p.to_json('/nas/home/yixiaoli/prompt_commonsense/Com2Sense/datasets/com2sense/test_t5.json', orient='records')

print("done!")

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-large and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0
['a computer', 'a pen']
pen is smaller than a computer mouse and weighs less than two ounces. when writing, it is easier to feel the pressure of the ink on the Meanwhile, compared with a pen, the computer is much more powerful. ipads can be as small as 0.3mm and weigh as little as 2g 
1
['nails', 'screws']
Compared with nails, screws are much stronger. compared with screws, they are a lot less expensive to use. Meanwhile, Compared with screws, nails are not as strong as screws. a nail can be as long as three times as thick as the head of the nail, depending on
2
['a taxi', '1 dollar']
compared with a taxi, 1 dollar is about the same price. if you're going to go to the beach, it's cheaper to rent  Meanwhile, compared with 1 dollar, a taxi fare is less than one dollar. fares for taxis in london are about the same as in the
3
["Daniel Cormier's merch", 'Conor McGregor']
conor mcGregor's merch isn't as expensive as daniel cormier. he has his own line of Meanwhile, merchandising for danie

pail is like a shovel - it can be used to sift through dirt and soil. Meanwhile, compared with a pail,a shovel is better. if you want to get rid of the sand in your garden, you need 
32
['a hair tie', 'a sock']
sock is better than a hair tie - it's softer and more breathable. Meanwhile, hair tie is a hair accessory. compared with sock, it's not as flimsy and bulky as you'd think 
33
['her foot', 'her mouth']
compared with her foot, her mouth is bigger. she's got a mouthful of sour cream in the back of her t-shirt  Meanwhile, compared with her mouth, her foot is the size of a golf ball. if she had stepped on it, it would have been like stepping on
34
['boat', 'plane']
Compared with boat, plane is faster and more comfortable. compared with plane, it takes less time to get from point a to b and vice versa Meanwhile, Compared with plane, boat is faster and more comfortable. compared with boat, it's a lot quieter and easier to get around on the water 
35
['peanuts', 'peanuts']
Compared with

compared with a boat, the piece of wood looks like wood. it's about the same size and weight as an average sized sailor' Meanwhile, boat is like a piece of furniture. compared with piece, it's about the same size as an average sized sailor’s house 
66
['a t-shirt', 'a bowtie']
compared with a t-shirt, the bowtie is more formal. bowties can be worn with or without the tie, depending on the occasion Meanwhile, bowtie, a t-shirt or sleeveless polo shirt?.
67
['his enclosure', 'his enclosure']
compared with his enclosure, he's in a much smaller enclosure. the only difference between the two is the size of the enclosure and the amount of space Meanwhile, compared with his enclosure, he's in a much smaller enclosure. the only difference between the two is the size of the enclosure and the amount of space
68
['his laptop', 'his phone']
compared with his laptop, his phone is a far cry. he uses his samsung galaxy note 4 as his primary phone - and it Meanwhile, compared with his phone, his lapto

compared with high end belongings, rental insurance protects your possessions. renters insurance is a good way to protect your rental property if you have  Meanwhile, compared with rental insurance, high end belongings are expensive to insure. if you're not careful, you could end up with a hefty
98
['more traditional parents', 'more makeup']
compared with more traditional parents, more makeup parents are more likely to have children. the average age of children born in the u.s. is 12 years old Meanwhile, compared with more make-up, more traditional parents have more makeup. 'traditional' parents are more likely to have a baby girl or boy in their first
99
['mouth', 'injection']
compared with mouth, injections are more effective than chewing. saline is less corrosive and has a longer half-life than  Meanwhile, Compared with injection, mouthwash is more effective than injection. it is also less expensive and has fewer side effects than a saline solution.
100
['a sprint', 'a marathon']
ma

compared with work, 10 MPH is 10 times more efficient. 10 mph is the equivalent of a ten-mile-per-hour road trip  Meanwhile, compared with 10 MPH, work speed is 10 times faster. work is more tiring, but it's not as bad as driving at 10 mph 
129
["Justin Bieber's new album", 'tomorrow']
compared with Justin bieber's new album, tomorrow will be a different story. tomorrow is the first day of the tour, which will take place in Meanwhile, compared with tomorrow, Justin bieber's new album is tomorrow. 'i wanna be you' is more of a pop-rock album 
130
['her legs', 'tomorrow night']
compared with her legs, tomorrow night's sarah williams' legs will be a lot shorter. Meanwhile, compared with tomorrow night, her legs are a lot shorter. she'll be able to see more of the pitch on tuesday night
131
['about ten minutes', 'several hours']
compared with about ten minutes, several hours is a lot of time. i'm not sure if it's worth it or not, but Meanwhile, Compared with several hours, about ten minute

compared with golf, soccer is the second most popular sport in the u.s. after golf. Meanwhile, compared with soccer, golf is the second most popular sport in the u.s. after soccer.
160
['home', 'his friends']
Compared with home, his friends at school are his best friends. he has a lot of friends in taiwan and thailand Meanwhile, Compared with his friends, home is a far cry from the party scene. he has no place to go to sleep, eat or sleep at night 
161
['her pregnancy', '18 months']
compared with her pregnancy, 18 months after giving birth, she is now a mother of two. mrs taylor's weight loss Meanwhile, compared with 18 months, her pregnancy was much shorter. she was able to conceive for the first time in april and july 
162
['3 days', '3 weeks']
compared with 3 days, 3 weeks, 1 month ago. a week is equal to one day of rest per day. Meanwhile, compared with 3 weeks, 3 days. a week is equivalent to 1 week, 1 day, and 2 days of rest.
163
['a week', 'tomorrow']
compared with a week ago, t

In [24]:
df_m = pd.read_json('/nas/home/yixiaoli/prompt_commonsense/Com2Sense/datasets/com2sense/m/test_m.json')
d = df_m[df_m['scenario']=='comparison']
d

In [ ]:
a = "his"
print(a[0:-1])